<a href="https://colab.research.google.com/github/sssuinlee/BDP_project/blob/main/tensor_rnn_1210.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
import pandas as pd

In [2]:
#csv 읽어오기
train = pd.read_csv("/content/drive/MyDrive/OTTO/train_prep.csv")
train = train.drop(train.columns[[0]],axis = 1)
display(train)

,customer_id,product_code,time_stamp,event_type
0,0,1517085,2022-07-31 22:00:00.025,clicks
1,0,1563459,2022-07-31 22:01:44.511,clicks
2,0,1309446,2022-08-01 15:23:59.426,clicks
3,0,16246,2022-08-01 15:28:39.997,clicks
4,0,1781822,2022-08-01 15:31:11.344,clicks
...,...,...,...,...
10285556,199999,928064,2022-08-01 06:46:48.967,clicks
10285557,199999,849970,2022-08-01 06:47:29.078,clicks
10285558,199999,1052480,2022-08-01 06:49:07.035,clicks
10285559,199999,487255,2022-08-01 06:49:21.116,clicks


In [3]:
#customer 0 의 세션
customer_0 = train[train.customer_id == 0]
display(customer_0)

,customer_id,product_code,time_stamp,event_type
0,0,1517085,2022-07-31 22:00:00.025,clicks
1,0,1563459,2022-07-31 22:01:44.511,clicks
2,0,1309446,2022-08-01 15:23:59.426,clicks
3,0,16246,2022-08-01 15:28:39.997,clicks
4,0,1781822,2022-08-01 15:31:11.344,clicks
...,...,...,...,...
271,0,843110,2022-08-28 10:58:18.768,clicks
272,0,938007,2022-08-28 10:59:15.390,clicks
273,0,1228848,2022-08-28 11:02:08.943,clicks
274,0,1740927,2022-08-28 11:09:02.173,clicks


In [4]:
#customer 0 의 orders 목록
customer_0_orders = customer_0[customer_0.event_type == 'orders']
display(customer_0_orders)

,customer_id,product_code,time_stamp,event_type
8,0,305831,2022-08-01 16:07:07.105,orders
9,0,461689,2022-08-01 16:07:07.105,orders
245,0,1199474,2022-08-26 22:29:00.651,orders
246,0,543308,2022-08-26 22:29:00.651,orders


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder

In [6]:
#OneHotEncoder 객체에 전체 데이터의 pid랑 etype 넣어서 fit
ohe = OneHotEncoder(handle_unknown="ignore")
pid_etype = pd.DataFrame(data={"product_code":train.product_code, "event_type":train.event_type})
ohe.fit(pid_etype)

OneHotEncoder(handle_unknown='ignore')

In [7]:
#OneHotEncoding 객체 column확인:[1, 3, 4, ..., 1855600, 1855601, carts, clicks, orders]
ohe.categories_

[array([      1,       3,       4, ..., 1855598, 1855600, 1855601]),
 array(['carts', 'clicks', 'orders'], dtype=object)]

In [8]:
#pid_etype_0 = customer 0 의 pid랑 etype
pid_etype_0 = pd.DataFrame(data={"product_code":customer_0.product_code, "event_type":customer_0.event_type})
display(pid_etype_0[:-1])

,product_code,event_type
0,1517085,clicks
1,1563459,clicks
2,1309446,clicks
3,16246,clicks
4,1781822,clicks
...,...,...
270,219925,clicks
271,843110,clicks
272,938007,clicks
273,1228848,clicks


In [9]:
pid_etype_0[-1:]

,product_code,event_type
275,161938,clicks


In [10]:
#encoded_0 = customer0의 pid랑 etype 을 원핫인코딩
encoded_0 = ohe.transform(pid_etype_0).toarray()

In [11]:
encoded_0.shape

(276, 951297)

In [ ]:
#원핫인코딩 풀었을때 
type(ohe.inverse_transform(encoded_0))

In [ ]:
"""
input_batch=[]
input_batch.append(encoded_0)
input_batch
"""

[array([[0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]])]

In [22]:
#한 세션 = 한 문장
#pid 1개 = 한 단어
#session(sentence) = "15512 5532 85811 11390 1002 3 52"
#session: pid_etype_0 같은거("pid":(...), "etype":(...))
#sessions: pid_etype_0 같은것들의 'list'. (단, type(sessions)=list)

customer_0 = train[train.customer_id == 0]
pid_etype_0 = pd.DataFrame(data={"product_code":customer_0.product_code, "event_type":customer_0.event_type})

customer_1 = train[train.customer_id == 1]
pid_etype_1 = pd.DataFrame(data={"product_code":customer_1.product_code, "event_type":customer_1.event_type})

customer_2 = train[train.customer_id == 2]
pid_etype_2 = pd.DataFrame(data={"product_code":customer_2.product_code, "event_type":customer_2.event_type})

sessions = [pid_etype_0, pid_etype_1, pid_etype_2]
#type(sessions) = list

In [23]:
print(pid_etype_0.shape, pid_etype_1.shape, pid_etype_2.shape)

(276, 2) (32, 2) (33, 2)


In [40]:
def batchmaker(sessions):
  input_batch = []
  target_batch = []

  for session in sessions:  #한 세션씩
     input_batch.append(ohe.transform(session[:20]).toarray())
     target_batch.append(ohe.transform(session[20:30]).toarray())

  return input_batch, target_batch

In [41]:
input_batch, target_batch = batchmaker(sessions)

In [42]:
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad = True)
##requires_grad=True 는 자동 미분기능.
##이 tensor 에 연산을 하면 계산 그래프가 생성
##backward 함수를 호출하면 그래프로부터 자동으로 미분이 계산됨. 
##왜 requires_grad=True??????????????????????????????

In [43]:
input_batch.shape
#3장짜리. 20개 행, 951297개 열. 

torch.Size([3, 20, 951297])

In [45]:
target_batch = torch.tensor(target_batch, dtype=torch.float32)

In [46]:
target_batch.shape
#3장짜리, 10개 행, 951297개 열. 

torch.Size([3, 10, 951297])

In [ ]:
batch_size = len(sessions)  #총 세션의 수
n_step = 20 #학습하려는 세션의 길이 (-1?)
set_size = len(ohe.categories_[0]) + len(ohe.categories_[1]) 


dtype = torch.float
##torch.nn.Module 상속. 
##nn.Module 은 .parameters(), .zero_grad() 같은 메소드를 가지고 있음. 

In [ ]:
#참고: https://tutorials.pytorch.kr/beginner/nn_tutorial.html
#      https://blog.naver.com/ghdals6864/222469230124 
class SessionRNN(nn.Module):
  def __init__(self):
    super(SessionRNN, self).__init__()

    self.rnn = nn.RNN(input_size=set_size, hidden_size=set_size, dropout=0.3)
    #input_size랑 hidden_size 약간이해안됨 왜 3이 아닌가요
    #dropout은 학습된 것을 잊어버리게 한다? =>0으로 해볼것...:https://wjddyd66.github.io/keras/Keras(4-2)/
    self.weights = nn.Parameter(torch.randn([set_size, set_size]).type(dtype))
    self.bias = nn.Parameter(torch.zeros(set_size))
    self.Softmax = nn.Softmax(dim=1)

  def forward(self, hidden, X):
    X = X.transpose(0, 1)
    outputs, hidden = self.rnn(X, hidden)
    outputs = outputs[-1]  # 최종 예측 Hidden Layer
    model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    return model

In [ ]:
##Training

model = SessionRNN()
criterion = nn.CrossEntropyLoss()   #손실함수 객체. 
###CrossEntropyLoss 는 softmax와 onehot 정답간의 loss를 계산한다. 
###손실을 무엇을 기준으로 계산할 것인지
optimizer = optim.Adam(model.parameters(), lr=0.01) 
#optimizer = optim.SGD([W, b], lr=0.01)   weight, bias, lr 지정해서 
#SGD나 Adam 대신 쓸 수 있는 함수들: https://seamless.tistory.com/38
#https://nuguziii.github.io/dev/dev-002/
#함수 정의가능


#참고: https://wikidocs.net/53560
for epoch in range(500):
  hidden = torch.zeros(1, batch_size, set_size, requires_grad=True)   #(가중치?)hidden 을 0으로 초기화. 학습으로 변경가능
  output = model(hidden, input_batch)     #input_batch 에 가중치 hidden을 써서 나온 output
  loss = criterion(output, target_batch)    #criterion(예측치, 답), loss 를 계산. 

  if (epoch + 1) % 100 == 0:      #99, 199, 299, 399, 499 번째 학습이면 loss 를 출력하도록
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
  optimizer.zero_grad()   #gradient(미분값)가 누적되지 않도록 0으로 초기화
  loss.backward()         #cost function 미분해서 gradient(현지점 기울기) 계산
  optimizer.step()        #weight, bias 업데이트

input = [session.split()[:2] for session in sessions]   #문장들의 앞 두 단어만 빼서 test input

hidden = torch.zeros(1, batch_size, set_size, requires_grad=True)   
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([session.split()[:2] for session in sessions], '->', [number_dict[n.item()] for n in predict.squeeze()])



In [21]:
torch.zeros(4, 3, 5, requires_grad=True)
##batch_size 는 총 세션 수: sample 에선 3이니까 3
##n_hidden 은 은닉층 사이즈..?

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]], requires_grad=True)

In [16]:
torch.randn(25, 4) 

tensor([[-0.6951, -0.6684,  0.6625,  1.2804],
        [ 0.7130, -0.6294,  2.3237, -0.4515],
        [-0.2377,  0.0238,  0.3350, -0.1720],
        [ 0.9591,  0.0650, -1.2027,  0.5548],
        [ 0.6534,  0.6527,  1.1423, -0.2141],
        [ 2.9274, -1.2261, -1.2072,  1.7172],
        [ 0.2958,  0.9260, -0.7736,  1.0029],
        [ 1.1614, -0.4391, -0.8090, -1.0658],
        [-0.2725, -0.9855,  0.0902,  0.9139],
        [ 1.8208,  0.2375,  0.2494,  0.8052],
        [ 0.7962,  0.5571,  1.4174, -0.3685],
        [ 0.8162,  1.1905, -0.9690,  0.0517],
        [-0.7742,  1.1767,  0.1789,  1.5995],
        [ 0.5040, -0.2066, -0.6987,  0.0123],
        [-0.0195, -1.2363,  0.8610,  0.2180],
        [-0.0363,  0.7766,  0.5888, -0.2472],
        [-0.8204, -1.0316, -0.5300,  0.1322],
        [ 1.0147, -0.2196,  0.5547,  0.0792],
        [ 0.4079,  0.5169,  1.0868, -1.1620],
        [-0.2893,  0.0363,  0.9100, -0.6976],
        [ 0.4923,  0.4882,  1.6414, -3.1770],
        [-0.1398,  0.4738,  0.4035

In [58]:
import math
(torch.randn(784,10)/math.sqrt(784)).shape

torch.Size([784, 10])

In [60]:
(torch.zeros(10)).shape

torch.Size([10])

In [49]:
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
dtype = torch.float

"""
Word Processing
"""
word_list = list(set(" ".join(sentences).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

word_dict

{'hate': 0,
 'you': 1,
 'milk': 2,
 'i': 3,
 'dog': 4,
 'like': 5,
 'coffee': 6,
 'love': 7,
 'cat': 8}